In [1]:
import numpy as np
import time
import pandas as pd

# Introducci&oacute;n

## Qu&eacute; es un CSP:
Un CSP esta definido por:
<ul>
    <li>
        Un estado de variables X que tienen posibles valores de un dominio D
    </li>
    <li>
        Un conjunto de restricciones las cuales nos indican las necesidades que tienen las variables para tomar sus valroes
    </li>
</ul>

## T&eacute;cnicas a explorar:

### Backtracking:

Esta tecnica prueba absolutamente todas las posibles combinaciones que pueda llegar a tener la soluci&oacute;n sin importar si son correctas o no.

Tiene una propiedad importante:

<ul>
    <li>
        No importa el orden en el que se pongan los valores
    </li>
</ul>

#### Las siguientes t&eacute;cnicas son h&eacute;uristicas para mejorar BackTracking

### MRV:

Esta t&eacute;cnica pretende seleccionar la variable con la menor cantidad de movimientos "Validos" posibles, de esta forma se busca poder la busqueda de manera temprana ya que si se encuentra una rama destinada a el fracaso se encuentra y se elimina rapidamente


### Degree:

Esta t&eacute;cnica pretende seleccionar primero las variables que tengan una mayor influencia en los siguentes movimientos.

### Least Constrataining Variable:

Esta t&eacute;cnica pretende seleccionar primero las variables que tengan una menor influencia en los siguentes movimientos dando una mayor flexibilidad a la obtenci&oacute;n de soluciones.


### Forward Checking:

Esta t&eacute;cnica pretende tener en mente los valores restantes de las variables sin asignar con forme se avanza en la busqueda.



## El problema de las N-reinas:
El problema de las n-reinas consiste en lo siguiente:
    <ul>
        En un tablero de nxn espacios encontrar una distribuci&oacute;n en la cual ninguna de las reinas se este atacando entre si.
    </ul>

    

# Clase base: n_queens

En esta clase se incluyen los m&eacute;todos mas importantes para las t&eacute;cnicas que se implementaran.

Los atributos de la clase incluyen el tablero (board), la cantidad de las reinas (n), un arreglo con las posiciones de las reinas (queens)

Los metodos son:

<ul>
        <li> getDiagonals: Esta funcion regresa todas las posiciones en diagonal a partir de una posicion dada </li>
        <li> getRows: Esta funcion regresa todas las posiciones en la misma fila a partir de una posicion dada</li>
    <li> getColumns: Esta funcion regresa todas las posiciones en columnas adyacentes a partir de una posicion dada  </li>
    <li>isSafe: Esta funcion determina si la reina analizada no esta siendo atacada por ninguna otra reina</li>
    <li>isGoal: Esta funcion determina si se tiene la cantidad N de reinas colocadas en el tablero y si ninguna se esta atacando entre si</li>
    <li>isValid: Esta funcion determina si el estado actual es valido determinando si todas y cada una de las reinas en el tablero estan a salvo</li>
</ul>

In [2]:
class n_queens:
    board = []
    n = None
    queens = None
    
    def __init__(self,n,board,queens = []):
        self.board = board
        self.n = n
        self.queens = queens
     
    def getDiagonals(self,pos):
        
        row,col = pos
        diag = []
        
        for i,j in zip(range(row,self.n),range(col,self.n)):
            if (i,j) != pos:
                diag.append((i,j))
        
        for i,j in zip(range(row,-1,-1),range(col,-1,-1)):
            if (i,j) != pos:
                diag.append((i,j))
                
        for i,j in zip(range(row,-1,-1),range(col,self.n)):
            if (i,j) != pos:
                diag.append((i,j))
       
        for i,j in zip(range(row,self.n),range(col,-1,-1)):
            if (i,j) != pos:
                diag.append((i,j))
        
        diag = sorted(diag,key = lambda x : x[0])
        return diag   
    
    def getRows(self,queen):
        rows = []
        row = queen[0]
        for i in range(self.n):
            rows.append((row,i))
        return rows

    def getColumns(self,queen):
        cols = []
        col = queen[1]
        for i in range(self.n):
            cols.append((i,col))
        
        return cols
    
    def isSafe(self,queen):

        
        for i in self.getRows(queen):
            if self.board[i] == 1 and i != queen:
                return False
        
        for i in self.getColumns(queen):
            if self.board[i] == 1 and i != queen:
                return False
        
        for i in self.getDiagonals(queen):
            if self.board[i] == 1 and i != queen:
                return False
        return True
    
    def isGoal(self):
        
        if len(self.queens) == self.n:
            for queen in self.queens:
                if not self.isSafe(queen):

                    return False
            return True
    
        
    def isValid(self):
        for queen in self.queens:
            if not self.isSafe(queen):
                return False
        return True

### BackTracking:

En esta t&eacute;cnica se consideran todos los casos, esta regresara la primera soluci&oacute;n que se encuentre.

La principal funci&oacute;n es la que genera vecinos ya que nos agrega todos los posibles valores que se puedan obtener a partir del estado actual.

In [3]:
class BackTracking(n_queens):
    neighbours = None
    
    def __init__(self,n,board,queens = []):
        n_queens.__init__(self,n,board,queens)
        #return None
    
    def getNeighbours(self):
        self.neighbours = []
        for i in range(self.n):
            for j in range(self.n):
                if not (i,j) in self.queens:
                    self.neighbours.append((i,j))


In [4]:
def executeBack(node):

    if not node.isValid():
        return False
    
    if node.isGoal():
        return node
    
    
    node.getNeighbours()
    
    for new in node.neighbours:
            board = node.board.copy()
            queens = node.queens[:]
            
            board[new] = 1
            queens.append(new)
            newobj = BackTracking(node.n,board,queens)
            
            final_node = executeBack(newobj)
            
            if final_node:
                return final_node
    return False

        

n = 4   
back = BackTracking(n,np.zeros((n,n)))
p = executeBack(back)
if p:
    print p.board


[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]


### MRV:

Esta t&eacute;cnica agrega un nuevo atributo a la clase padre llamado remaining el cual es un arreglo con las posiciones validas restantes en un punto determinado.

La diferencia entre esta heuristica y otras es que esta fue aplicada ordenando los vecinos en cuenstion a la cantidad de valores (posiciones disponibles) restantes.

#### Metodos nuevos:

<ul>
    <li>getRemainingValues: Esta funcion regresa las posiciones validas restantes considerando a todas las reinas ya colocadas en el tablero </li>
    <li>getNeighbours: Esta funcion obtiene todos los vecinos a partir de los valores restantes ordenandolos de manera ascendente en cuestion de cantidad </li>
</ul>

In [5]:
class MRV(BackTracking):
    remaining = None
    
    def __init__(self,n,board,queens = [],remaining = []):
        BackTracking.__init__(self,n,board,queens)
        self.remaining = None

    
    def getRemainingValues(self,pos):
        #Obtiene los valores que quedan
        #Tambien considera las reinas
        remaining = []
        moves = []
        for queen in self.queens:
            moves += self.getRows(queen) + self.getColumns(queen) +self.getDiagonals(queen)

        moves += self.getRows(pos) + self.getColumns(pos) +self.getDiagonals(pos)

        for i in range(self.n):
            for j in range(self.n):
                if not (i,j) in moves:
                    remaining.append((i,j))
        return remaining            


    def getNeighbours(self):
        self.neighbours = []

        for i in range(self.n):
            for j in range(self.n):
                if not (i,j) in self.queens:
                    board = self.board.copy()
                    queens = self.queens[:]
                    queens.append((i,j))
                    board[i,j] = 1
                    self.neighbours.append(((i,j),self.getRemainingValues((i,j)),board,queens))
        self.neighbours = sorted(self.neighbours,key = lambda n: len(n[1]))


In [6]:
def executeMRV(node):
    
    if not node.isValid():
        return False
    
    if node.isGoal():
        return node
    
    node.getNeighbours()

    for new in node.neighbours:
        
        newobj = MRV(node.n,new[2],new[3])
        final = executeMRV(newobj)
        
        if final:
            return final
    
p = MRV(n,np.zeros((n,n)))
print executeMRV(p).board
    

[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]


### Degree:
Agrega un nuevo atributo llamado constrataints el cual contiene las posiciones que no pueden ser usadas.

Esta heuristica se aplica con el ordenamiento de los siguientes vecinos el cual se basa en la cantidad de restricciones que obtienen los vecinos obteniendo de esta forma los mejores nodos siguientes

Metodos:

<ul>
    <li>getConstrataints: Esta funcion obtiene las posiciones que no pueden ser usadas en algun punto de la busqueda </li>
    <li>getNeighbourConstrataint: Esta funcion obtiene la cantidad de restricciones de los vecinos la cual es usada para el ordenamiento en otra funcion. </li>
    <li>getNeighbours: Esta funcion obtiene los vecinos validos restantes y los ordena de forma en la que aquel vecino que tenga una mayor infuencia en las restricciones. </li>
</ul>

In [7]:
class Degree(BackTracking):
    constrataints = None
    
    def __init__(self,n,board,queens = [],constrataints = []):
        BackTracking.__init__(self,n,board,queens)
        self.constrataints = constrataints
        
    def getConstrataints(self):
        
        moves = []
        for queen in self.queens:
            moves += self.getRows(queen) + self.getColumns(queen) +self.getDiagonals(queen)
        self.constrataints = moves
        
    def getNeighbourConstrataint(self,queen):
        constrataints = self.constrataints[:]
        constrataints += self.getRows(queen) + self.getColumns(queen) +self.getDiagonals(queen)
        return len(constrataints)
    
    def getNeighbours(self):
        self.neighbours = []
        self.getConstrataints()
        
        for i in range(self.n):
            for j in range(self.n):
                if not (i,j) in self.constrataints:
                    board = self.board.copy()
                    queens = self.queens[:]
                    
                    board[i,j] = 1
                    queens.append((i,j))
                    self.neighbours.append((board,queens,self.constrataints,self.getNeighbourConstrataint((i,j))))
        
        
        
        
        self.neighbours = sorted(self.neighbours,key = lambda x: x[3], reverse = True)

In [8]:
def executeDegree(node):
    
    if not node.isValid():
        return False
    
    if node.isGoal():
        return node
    
    node.getNeighbours()
    
    for new in node.neighbours:
        
        newobj = Degree(node.n,new[0],new[1],new[2])
        final = executeDegree(newobj)
        
        if final:
            return final

p = Degree(n,np.zeros((n,n)))
ans = executeDegree(p)
print ans.board

[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]


### LCV

Esta heuristica se aplica de forma contraria a Degree ya que se ordenan los vecinos de forma ascendente a la cantidad de restricciones que se obtienen ( La mayor cantidad de posiciones restantes en el tablero ).

Los siguientes metodos fueron modificados:

<ul>
    <li> getNeighbourConstrataints: Ahora regresa las restricciones </li>
    <li> getNeighbours: Los ordena de menor a mayor respecto a la cantidad de restricciones</li>
</ul>
    


In [9]:
class LCV(Degree):
    
    def __init__(self,n,board,queens = [],constrataints = []):
        Degree.__init__(self,n,board,queens,constrataints)
        
    def getNeighbourConstrataint(self,queen):
        constrataints = self.constrataints[:]
        constrataints += self.getRows(queen) + self.getColumns(queen) +self.getDiagonals(queen)
        return constrataints
        
    
    def getNeighbours(self):
        self.neighbours = []
        self.getConstrataints()
        
        for i in range(self.n):
            for j in range(self.n):
                if not (i,j) in self.constrataints:
                    board = self.board.copy()
                    queens = self.queens[:]
                    
                    board[i,j] = 1
                    queens.append((i,j))
                    self.neighbours.append((board,queens,self.getNeighbourConstrataint((i,j))))
        
        self.neighbours = sorted(self.neighbours,key = lambda x: len(x[2]))

In [10]:
def executeLCV(node):
    
    if not node.isValid():
        return False
    
    if node.isGoal():
        return node
    
    node.getNeighbours()
    
    for new in node.neighbours:
        #print new
        newobj = LCV(node.n,new[0],new[1],new[2])
        final = executeLCV(newobj)
        
        if final:
            return final
p = LCV(n,np.zeros((n,n)))
ans = executeLCV(p)
print ans.board

[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]


### ForwardChecking
Para esta heuristica se toma en cuenta el dominio de posibles valores que pueda obtener una variable.

Al igual que antes de entrar recursivamente a un nuevo nodo se verifica si tiene mas de un valor en el dominio de la nueva variable a evaluar.

Metodos:

<ul>
    <li> getRowDomain: Obtiene todos los valores posibles de la variable actual </li>
    <li> getRemaining: Obtiene los valores restantes de acuerdo a la reina actual</li>
    <li> CheckForward: Esta funcion determina si el dominio de la variable es correcto ( no se atacan las reinas ) </li>
</ul>


In [11]:
class ForwardChecking(Degree):
    
    def __init__(self,n,board,queens = []):
        BackTracking.__init__(self,n,board,queens)
    
    def getRowDomain(self,actual):
        rows = []
        for i in range(self.n):
            rows.append(i)
        return rows
    
    def getRemaining(self,actual):
        
        self.getConstrataints()
        remaining = []
        
        for i in range(self.n):
            for j in range(actual + 1,self.n):
                if not (i,j) in self.constrataints:
                    remaining.append((i,j))
        
        return remaining
    
    
    def CheckForward(self,row_actual,actual):
        
        domain  = self.getRowDomain(actual)
        domaincopy = list(domain)
        
        for row in domain:
            if not self.isSafe((row,actual)):
                domaincopy.remove(row)
                
        return len(domaincopy) == 0
        
    
    

### Ejecucion:

La ejecucion de este metodo es un tanto distinta a los demas:

<ol>
  <li>Determinar si el nodo actual es el nodo objetivo</li>
  <li>Determinar si ya llegamos al limite de nuestro tablero ( buscar en la culmna n+1 en un tablero de nxn )</li>
  <li>Para cada valor del dominio de la reina:
     <ul>
      <li>Creamos un nuevo nodo</li>
      <li>Verificar que el dominio de los valores restantes no este vacio en el caso de que sea asi se marca una bandera</li></ul>
    </li>
    <li>En caso de que la bandera no este marcada se ejecuta la funcion de nueva recursiva</li>
</ol>

In [12]:
def execForward(node,actual = 0):
    
    if node.isGoal():
        print node.board
        return True
    
    if actual >= node.n:
        
        return False
    
    
    for row in node.getRowDomain(actual):
            
        
        board = node.board.copy()
        queens = node.queens[:]
        
        board[row,actual] = 1
        queens.append((row,actual))
        
        newnode = ForwardChecking(node.n,board,queens)
        
        emptydomain = False
        
        for remain in newnode.getRemaining(actual):

            if newnode.CheckForward(remain[0],remain[1]):
                
                emptydomain = True
                break
                
        if not emptydomain:
            
            res = execForward(newnode,actual+1)
            
            if res:
                return res
      
        
        
    
n = 4

execForward(ForwardChecking(n,np.zeros((n,n))))

[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]


True

### Ejecucion de diferentes valores para N

In [29]:
def ejecutaTodo(n):
    mapa = np.zeros((n,n))
    
    print "\n----BackTracking----\n"
    stime = time.time()
    print executeBack(BackTracking(n,mapa)).board
    ftime = time.time()
    backtime = ftime-stime
    
    print "\n----MRV----\n"
    
    stime = time.time()
    print executeMRV(MRV(n,mapa)).board
    ftime = time.time()
    mrvtime = ftime-stime
    
    print "\n----Degree----\n"
    
    stime = time.time()
    print executeDegree(Degree(n,mapa)).board
    ftime = time.time()
    degreetime = ftime-stime
    
    print "\n----LCV----\n"
    
    stime = time.time()
    print executeLCV(LCV(n,mapa)).board
    ftime = time.time()
    lcvtime = ftime-stime
    
    print "\n----Forward Checking----\n"
    
    stime = time.time()
    execForward(ForwardChecking(n,mapa))
    ftime = time.time()
    forwardtime = ftime-stime
    
    indexes = ["Segs"]
    dataset = pd.DataFrame({"BackTracking": backtime,"MRV": mrvtime,"Degree": degreetime,"LCV": lcvtime,"Forward": forwardtime},index = indexes)
    
    return dataset

### N = 4

In [24]:
ejecutaTodo(4)


----BackTracking----

[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]

----MRV----

[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]

----Degree----

[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]

----LCV----

[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]

----Forward Checking----

[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]


,BackTracking,Degree,Forward,LCV,MRV
Segs,0.016442,0.008252,0.01637,0.005578,0.088298


### N = 5

In [25]:
ejecutaTodo(5)


----BackTracking----

[[1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]

----MRV----

[[0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]

----Degree----

[[0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]

----LCV----

[[1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]

----Forward Checking----

[[1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]]


,BackTracking,Degree,Forward,LCV,MRV
Segs,0.008488,0.00436,0.054556,0.006634,0.021945


### N = 6

In [26]:
ejecutaTodo(6)


----BackTracking----

[[0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]

----MRV----

[[0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]

----Degree----

[[0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]

----LCV----

[[0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]

----Forward Checking----

[[0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]]


,BackTracking,Degree,Forward,LCV,MRV
Segs,2.092128,1.993267,2.151795,0.498328,40.629163


### N = 7

In [27]:
ejecutaTodo(7)


----BackTracking----

[[1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]]

----MRV----

[[0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]]

----Degree----

[[0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]]

----LCV----

[[1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]]

----Forward Checking----

[[1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0.]]


,BackTracking,Degree,Forward,LCV,MRV
Segs,0.036529,0.03229,10.069257,0.270955,0.812313


### Conclusiones:

Todos los metodos tienen sus ventajas y desventajas, en lo personal Forward Checking fue aquel que rompio el "molde" ya que no fue tan sencillo de implementar comparado a los demas debido a que me tomo por sorpresa al intentar leer los pseudocodigos.

Las principales ventajas de usar una tecnica basada en DFS es que los valores son conmutables ( No importa el orden en el que se coloquen siempre dara el mismo resultado )

Sin embargo la principal desventaja de usar estas tecnicas es que puede que no encontremos la respuesta de la forma "optima".

Las heuristicas anteriores a Forward Checking fueron heuristicas que ayudaron a elegir el siguiente nodo que se exploraria de forma en la que pudiesemos podar el arbol de forma pronta ( MRV/Degree ) o mantener la mayor cantidad de espacos restantes en el tablero ( LCV )

Finalmente Forward Checking intenta verificar el dominio de las siguientes variables para que no tenga que verificar los nodos "muertos"

De acuerdo a mis pruebas, puedo determinar que en terminos generales las tecnicas mas "rapidas" son Degree y Least Constratained Values en los cuales fueron los que estuvieron menor tiempo en ejecucion.


